In [28]:
# Import Libraries
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from uncertainties import ufloat
import warnings


# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import parse_sample_pits


In [29]:
# Parse all snowpit files from the data folder
all_pits = parse_sample_pits('data')


Successfully parsed 50278 files
Failed to parse 0 files


In [32]:
# Collect Data
pit_info = []
temp_info = []

for pit in all_pits:
    pit_info_dict = {
        # User info
        'pit_id': pit.core_info.pit_id,
        # Weather
        'air_temp_value': pit.core_info.weather_conditions.air_temp_pres[0] if pit.core_info.weather_conditions.air_temp_pres else None,
        'air_temp_units': pit.core_info.weather_conditions.air_temp_pres[1] if pit.core_info.weather_conditions.air_temp_pres else None,
        'temp_profile_obs':len(pit.snow_profile.temp_profile)
    }
    pit_info.append(pit_info_dict)
    

    for obs in pit.snow_profile.temp_profile:
        temp_info_dict = {
            'pit_id': pit.core_info.pit_id,
            'depth_value': obs.depth[0] if obs.depth else None,
            'depth_units': obs.depth[1] if obs.depth else None,
            'temp_value': obs.snow_temp[0] if obs.snow_temp else None,
            'temp_units': obs.snow_temp[1] if obs.snow_temp else None
        }
        temp_info.append(temp_info_dict)


pit_df = pd.DataFrame(pit_info)
temp_df = pd.DataFrame(temp_info)

print(pit_df.head())
print(temp_df.head())


  pit_id  air_temp_value air_temp_units  temp_profile_obs
0  51686             NaN           None                 0
1   5847            -1.6           degC                 6
2   9544             NaN           None                18
3  30734           -13.0           degC                 7
4  43746             NaN           None                 0
  pit_id  depth_value depth_units  temp_value temp_units
0   5847          0.0          cm        -1.6       degC
1   5847         13.0          cm        -3.0       degC
2   5847         20.0          cm        -2.8       degC
3   5847         38.0          cm        -2.0       degC
4   5847         55.0          cm        -1.0       degC


In [33]:
# Confirm consistant units

print("Unique air temp units:")
print(pit_df['air_temp_units'].unique())

print("Unique temp depth units:")
print(temp_df['depth_units'].unique())

print("Unique temp units:")
print(temp_df['temp_units'].unique())


Unique air temp units:
[None 'degC']
Unique temp depth units:
['cm']
Unique temp units:
['degC']


In [38]:
# Summary stats for air temp

print(f"Number of pits: {len(all_pits)}")
pits_with_air_temp = pit_df['air_temp_value'].count()
print(f"Number of pits with air temp observations: {pits_with_air_temp}, {(pits_with_air_temp/len(all_pits)*100):.2f}%")

# Number of pits with temp profiles
pits_with_temp_profile = (pit_df['temp_profile_obs'] > 0).sum()
print(f"Number of pits with temp profiles: {pits_with_temp_profile}, {(pits_with_temp_profile/len(all_pits)*100):.2f}%")

# Number of temperature observations
print(f"Number of temperature profile observations: {len(temp_df)}")



Number of pits: 50278
Number of pits with air temp observations: 35747, 71.10%
Number of pits with temp profiles: 23138, 46.02%
Number of temperature profile observations: 213790


In [ ]:
# Summary stats and visualization for air temp


In [ ]:
# Summary stats and visualization for temp profile observations
